
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 2B -  Create Streaming Tables with SQL using Auto Loader

In this demonstration we will create a streaming table to incrementally ingest files from a volume using Auto Loader with SQL. 

When you create a streaming table using the CREATE OR REFRESH STREAMING TABLE statement, the initial data refresh and population begin immediately. These operations do not consume DBSQL warehouse compute. Instead, streaming table rely on serverless DLT for both creation and refresh. A dedicated serverless DLT pipeline is automatically created and managed by the system for each streaming table.

### Learning Objectives

By the end of this lesson, you should be able to:
- Create streaming tables in Databricks SQL for incremental data ingestion.
- Refresh streaming tables using the REFRESH statement.

### RECOMMENDATION

The CREATE STREAMING TABLE SQL command is the recommended alternative to the legacy COPY INTO SQL command for incremental ingestion from cloud object storage. Databricks recommends using streaming tables to ingest data using Databricks SQL. 

A streaming table is a table registered to Unity Catalog with extra support for streaming or incremental data processing. A DLT pipeline is automatically created for each streaming table. You can use streaming tables for incremental data loading from Kafka and cloud object storage.

## REQUIRED - SELECT CLASSIC COMPUTE

**NOTE: We'll use a classic compute cluster to set up the demonstration files, as Python is required. After that, you'll need to switch to a SQL warehouse to create the streaming tables using SQL.**

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

**TROUBLESHOOTING:** If you select a SQL Warehouse, an error will be returned since Python is used for the setup.

In [0]:
%run ./Includes/Classroom-Setup-Auto-Loader

## REQUIRED - SELECT YOUR SERVERLESS SQL WAREHOUSE

**NOTE: Creating streaming tables with Databricks SQL requires a SQL warehouse.**.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Before executing cells in this notebook, please select the **SHARED SQL WAREHOUSE** in the lab. Follow these steps:

1. Navigate to the top-right of this notebook and click the drop-down to select compute (it might say **Connect**). Complete one of the following below:

   a. Under **Recent resources**, check to see if you have a **shared_warehouse SQL**. If you do, select it.

   b. If you do not have a **shared_warehouse** under **Recent resources**, complete the following:

    - In the same drop-down, select **More**.

    - Then select the **SQL Warehouse** button.

    - In the drop-down, make sure **shared_warehouse** is selected.

    - Then, at the bottom of the pop-up, select **Start and attach**.

</br>
   <img src="./Includes/images/sql_warehouse_xsmall.png" alt="SQL Warehouse" width="600">

2. Run the following cell to configure your working environment for this notebook.

    **NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-SQL-Auto-Loader

3. View the default catalog and schema. Confirm the default catalog is **dbacademy** and the default schema is your **labuser** schema.

In [0]:
SELECT current_catalog(), current_schema()

## B. Create Streaming Tables for Incremental Processing

1. Complete the following to explore the volume `/Volumes/dbacademy/your-lab-user-schema/csv_files_autoloader_source` and confirm it contains a single CSV file.

   a. Select the catalog icon on the left ![Catalog Icon](./Includes/images/catalog_icon.png).

   b. Expand the **dbacademy** catalog.

   c. Expand your **labuser** schema.

   d. Expand **Volumes**.

   e. Expand the **csv_files_autoloader_source** volume.

   f. Confirm it contains a single CSV file named **000.csv**.

2. Run the query below to view the data in the CSV file(s) in your cloud storage location. Notice that it was returned in tabular format and contains 3,149 rows.

In [0]:
SELECT *
FROM read_files(
  '/Volumes/dbacademy/' || DA.schema_name || '/csv_files_autoloader_source',
  format => 'CSV',
  sep => '|',
  header => true
);


#### Create a STREAMING TABLE using Databricks SQL
3. Your goal is to create an incremental pipeline that only ingests new files (instead of using traditional batch ingestion). You can achieve this by using [streaming tables in Databricks SQL](https://docs.databricks.com/aws/en/dlt/dbsql/streaming) (Auto Loader).

   - The SQL code below creates a streaming table that will be scheduled to incrementally ingest only new data every week. 
   
   - A pipeline is automatically created for each streaming table. You can use streaming tables for incremental data loading from Kafka and cloud object storage.

   **NOTE:** Incremental batch ingestion automatically detects new records in the data source and ignores records that have already been ingested. This reduces the amount of data processed, making ingestion jobs faster and more efficient in their use of compute resources.

   **REQUIRED: Please insert the path of your csv_files_autoloader_source volume in the `read_files` function. This process will take about a minute to run and set up the incremental ingestion pipeline.**

In [0]:
-- YOU WILL HAVE TO REPLACE THE EXAMPLE PATH BELOW WITH THE PATH TO YOUR csv_file_autoloader_source VOLUME.
-- You can find the volume in your navigation bar on the right and insert the path
-- OR you can replace `your-labuser-name` with your specific labuser name (name of your schema)

CREATE OR REFRESH STREAMING TABLE sql_csv_autoloader
SCHEDULE EVERY 1 WEEK     -- Scheduling the refresh is optional
AS
SELECT *
FROM STREAM read_files(
  '/Volumes/dbacademy/your-labuser-name/csv_files_autoloader_source',  -- Insert the path to you csv_files_autoloader_source volume (example shown)
  format => 'CSV',
  sep => '|',
  header => true
);

4. Complete the following to view the streaming table in your catalog.

   a. Select the catalog icon on the left ![Catalog Icon](./Includes/images/catalog_icon.png).

   b. Expand the **dbacademy** catalog.

   c. Expand your **labuser** schema.

   d. Expand your **Tables**.

   e. Find the **sql_csv_autoloader** table. Notice that the Delta streaming table icon is slightly different from a traditional Delta table:
    
    ![Streaming table icon](./Includes/images/streaming_table_icon.png)

5. Run the cell below to view the streaming table. Confirm that the results contain **3,149 rows**.

In [0]:
SELECT *
FROM sql_csv_autoloader;

6. Describe the STREAMING TABLE and view the results. Notice the following:

- Under **Detailed Table Information**, notice the following rows:
  - **View Text**: The query that created the table.
  - **Type**: Specifies that it is a STREAMING TABLE.
  - **Provider**: Indicates that it is a Delta table.

- Under **Refresh Information**, you can see specific refresh details. Example shown below:

##### Refresh Information

| Field                   | Value                                                                                                                                         |
|-------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------|
| Last Refreshed          | 2025-06-17T16:12:49.168Z                                                                                                                      |
| Last Refresh Type       | INCREMENTAL                                                                                                                                   |
| Latest Refresh Status   | Succeeded                                                                                                                                     |
| Latest Refresh          | https://example.url.databricks.com/#joblist/pipelines/bed6c715-a7c1-4d45-b57c-4fdac9f956a7/updates/9455a2ef-648c-4339-b61e-d282fa76a92c (this is the path to the Declarative Pipeline that was created for you)|
| Refresh Schedule        | EVERY 1 WEEK                                                                                                                                 |

In [0]:
DESCRIBE TABLE EXTENDED sql_csv_autoloader;

7. The `DESCRIBE HISTORY` statement displays a detailed list of all changes, versions, and metadata associated with a Delta streaming table, including information on updates, deletions, and schema changes.

    Run the cell below and view the results. Notice the following:

    - In the **operation** column, you can see that a streaming table performs three operations: **CREATE TABLE**, **DLT SETUP** and **STREAMING UPDATE**.
    
    - Scroll to the right and find the **operationMetrics** column. In row 1 (Version 2 of the table), the value shows that the **numOutputRows** is 3149, indicating that 3149 rows were added to the **sql_csv_autoloader** table.

In [0]:
DESCRIBE HISTORY sql_csv_autoloader;

8. Complete the following steps to manually add another file to your cloud storage location:  
   `/Volumes/dbacademy/your-lab-user-schema/csv_files_autoloader_source`.

   a. Click the catalog icon on the left ![Catalog Icon](./Includes/images/catalog_icon.png).

   b. Expand the **dbacademy** catalog.

   c. Expand your **labuser** schema.

   d. Expand **Volumes**.

   e. Open the **auto_loader_staging_files** volume.

   f. Right-click on the **001.csv** file and select **Download volume file** to download the file locally.

   g. Upload the downloaded **001.csv** file to the **csv_files_autoloader_source** volume:

      - Right-click on the **csv_files_autoloader_source** volume. 

      - Select **Upload to volume**.  

      - Choose and upload the **001.csv** file from your local machine.

   h. Confirm your volume **csv_files_autoloader_source** contains two CSV files (**000.csv** and **001.csv**).


    **NOTE:** Depending on your laptop’s security settings, you may not be able to download files locally.


9. Next, manually refresh the STREAMING TABLE using `REFRESH STREAMING TABLE table-name`. 

- [Refresh a streaming table](https://docs.databricks.com/aws/en/dlt/dbsql/streaming#refresh-a-streaming-table) documentation

    **NOTE:** You can also go back to **Create a STREAMING TABLE using Databricks SQL (direction number 3)** and rerun that cell to incrementally ingest only new files. Once complete come back to step 8.

In [0]:
REFRESH STREAMING TABLE sql_csv_autoloader;

10. Run the cell below to view the data in the **sql_csv_autoloader** table. Notice that the table now contains **6,081 rows**.


In [0]:
SELECT *
FROM sql_csv_autoloader;

11. Describe the history of the **sql_csv_autoloader** table. Observe the following:

  - Version 3 of the streaming table includes another **STREAMING UPDATE**.

  - Expand the **operationMetrics** column and note that only **2,932 rows** were incrementally ingested into the table from the new **001.csv** file.


In [0]:
DESCRIBE HISTORY sql_csv_autoloader;

12. Drop the streaming table.

In [0]:
DROP TABLE IF EXISTS sql_csv_autoloader;

## Additional Resources

- [Streaming Tables Documentation](https://docs.databricks.com/gcp/en/dlt/streaming-tables)

- [CREATE STREAMING TABLE Syntax](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-create-streaming-table)

- [Using Streaming Tables in Databricks SQL](https://docs.databricks.com/aws/en/dlt/dbsql/streaming)

- [REFRESH (MATERIALIZED VIEW or STREAMING TABLE)](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-refresh-full)

- [COPY INTO (legacy)](https://docs.databricks.com/aws/en/ingestion/#copy-into-legacy)

- [Lakeflow Declarative Pipelines](https://docs.databricks.com/aws/en/dlt/)
---

#### BONUS Material: Course Appendix

In the course **Appendix** folder, you'll find a demonstration using Python Auto Loader in the **A2 - Python Auto Loader** notebook. This is extra material that you can explore outside of class.

&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>